# Квадратурные формулы типа Гаусса
### Вычислить интеграл с заданной точностью, предварительно определив число узлов и коэффициенты
$\int\limits_0^1\sqrt{x}\cos x^2dx,\:\varepsilon=10^{-4}.$

#### Чтобы промежуток интегрирования стал $[-1;\:1]$, сделаем замену переменной $x = \dfrac{1 - t}{2},$ тогда

$\int\limits_0^1\sqrt{x}\cos x^2dx=\frac{\sqrt{2}}{4}\int\limits_{-1}^1\sqrt{1-x}\cos\left(\frac{1-x}{2}\right)^2dx.$

In [1]:
from IPython.display import display, Latex
import math


def legendre(n):  # builds Legendre's polynomial of degree n recursively
    if n == 0:
        return lambda x: 1
    if n == 1:
        return lambda x: x
    return lambda x: ((2 * n) - 1) * x * legendre(n - 1)(x) / n - (n - 1) * legendre(n - 2)(x) / n


def legendre_der(n):  # builds polynomial's derivative
    if n == 0:
        return lambda x: 0
    if n == 1:
        return lambda x: 1
    return lambda x: (n / (1 - x ** 2)) * (legendre(n - 1)(x) - x * legendre(n)(x))


def get_legendre_roots(n, accur=1e-6):  # find roots of polynomial
    roots = []
    pol = legendre(n)
    d_pol = legendre_der(n)
    for i in range(1, n + 1):
        x0 = math.cos(math.pi * (4 * i - 1) / (4 * n + 2))
        while True:
            x1 = x0 - pol(x0) / d_pol(x0)
            if abs(x1 - x0) <= accur:
                roots.append(x1)
                break
            x0 = x1
    return roots


def quadrature(func, n):  # calculates Gaussian quadrature for given function func with n knots
    roots = get_legendre_roots(n)
    d_pol = legendre_der(n)
    quadr = 0
    for i in range(1, n + 1):
        coeff = 2 / ((1 - roots[i - 1] ** 2) * (d_pol(roots[i - 1]) ** 2))
        quadr += coeff * func(roots[i - 1])
    return quadr

#### Полиномы Лежандра степени $n$ вычисляются по рекуррентной формуле:
$P_n(x)=\frac{2n-1}{n}xP_{n-1}(x)-\frac{n-1}{n}P_{n-2}(x),\:n\ge2;\:\:P_0(x)=1,\:\: P_1(x)=x.$

#### Производную полинома можно найти с помощью полиномов меньшей степени:
$P_n'(x)=\frac{n}{1-x^2}\left(P_{n-1}(x)-xP_n(x)\right).$

#### Корни полинома Лежандра находятся по методу Ньютона с помощью следующих соотношений:
$x_i^{(k+1)}\approx x_i^{(k)}-\frac{P_n(x_i^{(k)})}{P_n'(x_i^{(k)})}.$

#### По корням полинома Лежандра строится квадратурная формула Гаусса:
$\int\limits_{-1}^1f(x)dx\approx\sum\limits_{k=1}^nA_kf(x_k),$
где коэффициенты $A_k$ вычисляются по формуле
$A_k=\frac{2}{(1-x_k^2)((P_n'(x_k))^2},\:k=1,\:\ldots,\:n.$

In [2]:
accuracy = 1e-4
f = lambda x: math.sqrt(2 - 2 * x) * math.cos(((x - 1) / 2) ** 2) / 4

k = 2
while True:
    q1 = quadrature(f, k)
    q2 = quadrature(f, k + 1)
    display(Latex("$\int\limits_0^1\sqrt{x}\cos x^2dx\\approx %.7f$, $n=%i$"%(q1, k)))
    display(Latex("$\int\limits_0^1\sqrt{x}\cos x^2dx\\approx %.7f$, $n=%i$"%(q2, k + 1)))
    if abs(q1 - q2) < accuracy:
        break
    k += 2
m = k + 1

display(Latex("В итоге $\int\limits_0^1\sqrt{x}\cos x^2dx\\approx %.7f$, точность достигается при $n=%i$"%(q2, k + 1)))

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

### Далее сравним полученные данные с формулами трапеций и Симпсона

In [3]:
def gen_f(a: float, b: float, n: float, func):
    h = abs(b - a) / n
    for i in range(n + 1):
        yield func(a + i * h)


def trapeze(a: float, b: float, n: float, func):
    g = gen_f(a, b, n, func)
    res = 0
    for p in g:
        res += 2 * p
    res -= func(a) + func(b)
    res *= (b - a) / (2 * n)
    return res


def simpson(a: float, b: float, n: float, func):
    g = gen_f(a, b, n, func)
    res = 0
    c = True
    for p in g:
        if c:
            res += 2 * p
            c = False
        else:
            res += 4 * p
            c = True
    res -= func(a) + func(b)
    res *= (b - a) / (3 * n)
    return res

### Формула трапеций:

In [4]:
a = -1
b = 1
eps = 1e-4  # given accuracy

k = 3
t1 = trapeze(a, b, k, f)
t2 = trapeze(a, b, k * 2, f)
while abs(t1 - t2) >= eps:
    k *= 2
    t1 = trapeze(a, b, k, f)
    t2 = trapeze(a, b, k * 2, f)
display(Latex("$\int\limits_0^1\sqrt{x}\cos x^2dx\\approx %.7f,\:$"
              "при $n=%i;$"%(t2, k * 2)))

<IPython.core.display.Latex object>

### Формула Симпсона:

In [5]:
k = 2
t1 = simpson(a, b, k, f)
t2 = simpson(a, b, k + 2, f)
while abs(t1 - t2) >= eps:
    k *= 2
    t1 = simpson(a, b, k, f)
    t2 = simpson(a, b, k * 2, f)
display(Latex("$\int\limits_0^1\sqrt{x}\cos x^2dx\\approx %.7f,\:$"
              "при $\:n=%i;$"%(t2, k * 2)))

<IPython.core.display.Latex object>

### Квадратурные формулы Гаусса:

In [6]:
display(Latex("$\int\limits_0^1\sqrt{x}\cos x^2dx\\approx %.7f$, $n=%i.$"%(q2, m)))

<IPython.core.display.Latex object>

Квадратурные формулы, очевидно, выигрывают в достижении необходимой точности, так она достигается при $n\le 10$, в
отличие от формул трапеции и Симпсона, где $n$ исчисляется сотнями.